## 自编码器简介
**传统机器学习任务很大程度上依赖于好的特征工程**，比如对数值型、日期时间型、种类型等特征的提取。特征工程往往是非常耗时耗力的，在图像、语音和视频中提取到有效的特征就更难了，工程师必须在这些领域有非常深入的理解，并且使用专业算法提取这些数据的特征。**深度学习则可以解决人工难以提取有效特征的问题，它可以大大缓解机器学习特征对特征工程的依赖**。深度学习在早期一度被认为是一种无监督的特征学习，**模仿了人脑对特征逐层抽象提取的过程**。这其中有两点很重要：
1. 无监督学习，即我们不需要标注数据就可以对数据进行一定程度的学习，这种学习是对数据内容的组织形式的学习，提取的是频繁出现的特征
2. 逐层抽象，特征是需要不断抽象的，就像人总是从简单基础的概念开始学习，再到复杂的概念。

自编码器(AutoEncoder),顾名思义，即可以使用自身的高阶特征编码自己。编码器其实也是一种神经网络，它的输入和输出是一致的，它借助稀疏编码的思想。目标是使用系数的一些高阶特征重新组合来重构自己。因此，**它的特点非常明显；第一，期望输入／输出一致；第二，希望使用高阶特征来重构自己，而不只是复制像素点。**

[参考 Hinton教授的文章 reducing the dimensionality of data with neural networks](http://science.sciencemag.org/content/313/5786/504)

hinton还提出了机遇深度信念网络(deep belief networks,dbn,由多层rbm堆叠而成)可使用无监督的逐层训练的贪心算法，为训练很深的网络提供了一个可行方案：我们可能很难直接训练极深的网络，但是可以用无监督的逐层训练提取特征，将网络的圈中初始化到一个比较好的位置，辅助后面的监督训练。无监督的逐层训练，其思想和自编码器非常相似。

In [0]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#模型权重的初始化器
def xavier_init(fan_in, fan_out, constant=1):
  low = -constant * np.sqrt(6.0/(fan_in+fan_out))
  high = constant*np.sqrt(6.0/(fan_in+fan_out))
  return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

In [0]:
#去噪自编码器类
class AdditiveGaussianNoiseAutoencoder(object):
  def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus,optimizer=tf.train.AdamOptimizer(), scale=0.1):
    self.n_input = n_input #输入变量数
    self.n_hidden = n_hidden #隐藏层节点数
    self.transfer = transfer_function #隐藏层激活函数
    self.scale = tf.placeholder(tf.float32)
    self.training_scale = scale
    network_weights = self._initialize_weights() #参数初始化函数
    self.weights=network_weights
    
    #定义网络结构
    
    #0.输入层。
    self.x = tf.placeholder(tf.float32, [None,self.n_input]) #n_input维的输入x
    
    #1.隐藏层。先将输入x加入噪声，即self.x+scale*tf.random_normal((n_input,))一个能提取特征的隐藏层
    self.hidden = self.transfer(tf.add(tf.matmul(self.x+scale*tf.random_normal((n_input,)), self.weights['w1']), self.weights['b1']))
    
    #2、输出层。经过隐藏层后，我们需要在输出层进行数据复原、重建操作(即建立reconstruction层)，即直接将隐藏层的输出self.hidden乘上输出层的权重w2，再加上输出层的偏置b2即可。
    self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
    
    #3、定义损失函数。直接使用平方误差，直接计算输入和输出之间的平方误差
    self.cost = 0.5*tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
    
    #4、定义优化器
    self.optimizer = optimizer.minimize(self.cost) #优化器
    
    #创建session，并初始化自编码器的全部模型参数
    init = tf.global_variables_initializer()
    self.sess = tf.Session()
    self.sess.run(init)
    
  #参数初始化函数
  def _initialize_weights(self):
    all_weights = dict()
    all_weights['w1'] = tf.Variable(xavier_init(self.n_input,self.n_hidden))
    all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],dtype=tf.float32))
    #输出层也就是重建层没有使用激活函数，不需要使用xavier进行参数初始化，直接初始化为0
    all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input], dtype=tf.float32))
    all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))
    
    return all_weights
  
  def partial_fit(self, X):
    #功能:用一个batch数据进行训练并返回当前损失cost
    cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x:X, self.scale:self.training_scale})
    return cost
  
  def calc_total_cost(self,X):
    #功能：在自编码器训练完毕后，在测试集上对模型性能进行测评时会用到的，它不会像partial_fit那样触发训练操作
    return self.sess.run(self.cost, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def transform(self, X):
    #功能:返回自编码器隐藏层的输出结果，目的是提供一个接口来获取抽象后的特征，自编码器的隐藏层的最主要功能就是学习出数据中的高阶特征
    return self.sess.run(self.hidden, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def generate(self, hidden=None):
    #功能：将隐藏层的输出结构作为输入，通过之后的重建层将提取到的高阶特征复原为原始数据。
    if hidden is None:
      hidden = np.random.normal(size=self.weights['b1'])
      
    return self.sess.run(self.reconstruction, feed_dict={self.hidden:hidden})
  
  def reconstruct(self, X):
    #功能:它整体运行一遍复原过程，包括提取高阶特征和通过高阶特征复原数据，即包括transform和generate两块
    return self.sess.run(self.reconstruction, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def getWeights(self):
    #功能:获取隐藏层的权重w1
    return self.sess.run(self.weights['w1'])
  
  def getBiases(self):
    #功能:获取隐藏层的偏置系数b1
    return self.sess.run(self.weights['b1'])

总结：transform、generate等函数，属于计算图中的子图。

In [0]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [0]:
#数据集进行标准化缩放
def standard_scale(X_train, X_test):
  preprocessor = prep.StandardScaler().fit(X_train)
  X_train = preprocessor.transform(X_train)
  X_test = preprocessor.transform(X_test)
  return X_train, X_test

In [0]:
#放回抽样，获取随机block数据的函数
def get_random_block_from_data(data, batch_size):
  start_index = np.random.randint(0, len(data)-batch_size)
  return data[start_index:(start_index+batch_size)]

In [0]:
#数据集缩放
X_train,X_test = standard_scale(mnist.train.images, mnist.test.images)

In [0]:
print(X_train.shape)
print(X_test.shape)

In [0]:
n_samples = int(mnist.train.num_examples)
training_epochs=20 #最大训练的轮数
batch_size = 128
display_step = 1

In [0]:
#创建一个AGN自编码器的实例，定义模型输入节点数n_input为784，自编码器的隐含层节点数n_hidden为200，隐藏层的激活函数transfer_function为softplus
#优化器optimizer为Adam且学习速率为0.001，同时将噪声的系数scale设为0.01

autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,n_hidden=200,transfer_function=tf.nn.relu,
                                              optimizer=tf.train.AdamOptimizer(learning_rate=0.001),scale=0.01)

In [0]:
#开始训练，在每一轮epoch循环开始时，将平均损失设为0，并计算总共需要的batch数
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch=int(n_samples/batch_size)
  for i in range(total_batch):
    batch_xs = get_random_block_from_data(X_train, batch_size)
    
    cost = autoencoder.partial_fit(batch_xs)
    avg_cost += cost/n_samples*batch_size
    
  if epoch % display_step ==0:
    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

In [0]:
print("Total cost:"+str(autoencoder.calc_total_cost(X_test)))

## 对比一下原图像以及使用自编码器复原的图像的区别

In [0]:
#对比原图像，已经使用编码器复原的图像之间的对比

#1、matplotlib显示出这个图像
x = mnist.train.images[0]
img = np.array(x.tolist()).reshape(28,28)
plt.imshow(img, cmap='gray')
plt.show()

In [0]:
#2、使用encoder把上面的图像编码成隐结构
latent_x = autoencoder.transform(x.reshape((1,784)))
print(latent_x.shape)

In [0]:
#3、使用reconstruction进行解码复原
x = autoencoder.generate(latent_x)
print(x.shape)

In [0]:
#4、使用matplotlib显示复原后的图像
img = np.array(x.tolist()).reshape(28,28)
plt.imshow(img, cmap='gray')
plt.show()

## 未完待续
后面需要提高神经网络的性能，多加一层隐藏层